In [9]:
import os
import time
import sys
import h5py
import re
import tensorflow  as tf
import numpy       as np
import pandas      as pd
from matplotlib    import pyplot as plt 
from os            import path
from tqdm          import tqdm

print("="*50) 
print(" TensorFlow version: {}".format(tf.__version__))
print(" Eager execution: {}".format(tf.executing_eagerly()))

 TensorFlow version: 2.5.0
 Eager execution: True


In [10]:
WORKSPACE_PATH      = os.environ['WORKSPACE_PATH']
SurQCTFldr          = WORKSPACE_PATH + '/SurQCT/surqct/'

RatesType           = 'KExcit'
ExcitType           = 'KInel'
NNRunIdx            = 12

PathToRunFld        = SurQCTFldr + '/../' + RatesType + '/all_temperatures/' + ExcitType + '/' 

TTranVec            = [1500.0, 5000.0, 10000.0, 15000.0, 20000.0]
ReadPlato           = False

In [11]:
print("\n[SurQCT]: Loading Modules and Functions ...")

sys.path.insert(0, SurQCTFldr  + '/src/Reading/')
from Reading  import read_levelsdata, read_diatdata, read_groupsmapping, read_sampledinitiallevels

InputFld = PathToRunFld + '/Run_' + str(NNRunIdx) + '/'
print(InputFld)
sys.path.insert(0, InputFld)


[SurQCT]: Loading Modules and Functions ...
/home/maitreyee/WORKSPACE//SurQCT/surqct//../KExcit/all_temperatures/KInel//Run_12/


In [12]:
print("\n[SurQCT]: Keep Loading Modules and Functions...")
from SurQCT_Input import inputdata

print("\n[SurQCT]: Initializing Input ...")
InputData    = inputdata(WORKSPACE_PATH, SurQCTFldr)

Prefix                    = 'Run_'
InputData.NNRunIdx        = NNRunIdx
InputData.PathToRunFld    = InputData.PathToRunFld+'/'+Prefix+str(InputData.NNRunIdx)
InputData.PathToFigFld    = InputData.PathToRunFld+'/'+InputData.PathToFigFld
InputData.PathToParamsFld = InputData.PathToRunFld+'/'+InputData.PathToParamsFld
InputData.PathToDataFld   = InputData.PathToRunFld+'/Data/'                                                               
InputData.PathToParamsFld = InputData.PathToRunFld+'/Params/' 
InputData.KineticFldr     = WORKSPACE_PATH+'/Air_Database/Run_0D_surQCT/database/kinetics/O3_UMN_Run'+str(NNRunIdx)+'/'
InputData.PathtoHDF5NN    = WORKSPACE_PATH  + '/Air_Database/HDF5_Database_NN/O3_UMN.hdf5'


[SurQCT]: Keep Loading Modules and Functions...

[SurQCT]: Initializing Input ...


In [13]:
print("\n[SurQCT]: Loading Final Modules ... ")

sys.path.insert(0, SurQCTFldr  + '/src/Model/' + InputData.ApproxModel + '/')
from Model import model


InputData.DefineModelIntFlg  = 0
InputData.TrainIntFlg        = 0
InputData.TransferFlg        = False
NN_KExcit                    = model(InputData, InputData.PathToRunFld, None, None)
NN_KExcit.load_params(InputData.PathToParamsFld)


[SurQCT]: Loading Final Modules ... 
[SurQCT]:   Variables Selected for Training:
[SurQCT]:     x =  ['EVib_i', 'ERot_i', 'rMin_i', 'rMax_i', 'VMin_i', 'VMax_i', 'Tau_i', 'ri_i', 'ro_i', 'TTran_i', 'EVib_Delta', 'ERot_Delta', 'rMin_Delta', 'rMax_Delta', 'VMin_Delta', 'VMax_Delta', 'Tau_Delta', 'ri_Delta', 'ro_Delta', 'TTran_Delta']
[SurQCT]:     y =  KExcit
[SurQCT]:   Loading ML Model from Folder: /home/maitreyee/WORKSPACE//SurQCT/surqct//../KExcit/all_temperatures/KInel//Run_12/MyModel
[SurQCT]:   Restoring ML Model Weigths to File: /home/maitreyee/WORKSPACE//SurQCT/surqct//../KExcit/all_temperatures/KInel//Run_12/MyModel/MyWeights.h5
[SurQCT]:   Loading ML Model Parameters from File:  /home/maitreyee/WORKSPACE//SurQCT/surqct//../KExcit/all_temperatures/KInel//Run_12/Params//ModelCheckpoint/cp-8534.ckpt


In [14]:
OtherVar           = InputData.OtherVar
xVarsVec_i         = InputData.xVarsVec_i + ['vqn','jqn']
xVarsVec_Delta     = InputData.xVarsVec_Delta
xVarsVec           = list(set(xVarsVec_i) | set(xVarsVec_Delta))
print('[SurQCT]:   Reading Variables: ', xVarsVec)

InputData.MultFact = 1.e+9
MinValueTrain      = 1.e-16 * InputData.MultFact
MinValueTest       = 1.e-16 * InputData.MultFact
NoiseSD            = 1.e-13 * InputData.MultFact

NMolecules         = len(InputData.PathToLevelsFile)

[SurQCT]:   Reading Variables:  ['VMax', 'jqn', 'vqn', 'VMin', 'rMax', 'Tau', 'rMin', 'ri', 'EVib', 'ro', 'ERot']


In [15]:
LevelsData = []
DiatData   = []
NLevels    = []
for iMol in range(NMolecules):

    LevelsDataTemp = read_levelsdata(InputData.PathToLevelsFile[iMol], xVarsVec, '')
    LevelsData.append(LevelsDataTemp)

    DiatDataTemp = read_diatdata(InputData.PathToDiatFile[iMol], InputData.Molecules[iMol], np.array(TTranVec), np.array(TTranVec))
    DiatData.append(DiatDataTemp)
    
    NLevelsTemp    = LevelsDataTemp.shape[0]
    NLevels.append(NLevelsTemp)
    
GroupMap = read_groupsmapping(InputData.PathToGrouping)

[SurQCT]:   Reading Molecular Levels Data from: /home/maitreyee/WORKSPACE//Air_Database/Run_0D/database/levels/46DPM_Sampled/O2_LogELogR.csv
[SurQCT]:   Reading Molecular Levels Data from: /home/maitreyee/WORKSPACE//CoarseAIR/coarseair/dtb/Molecules/O2/UMN/FromUMN_Sorted.inp
[SurQCT]:   Reading Molecular Levels Data from: /home/maitreyee/WORKSPACE//Air_Database/Run_0D/database/levels/46DPM_Sampled/O2_LogELogR.csv
[SurQCT]:   Reading Molecular Levels Data from: /home/maitreyee/WORKSPACE//CoarseAIR/coarseair/dtb/Molecules/O2/UMN/FromUMN_Sorted.inp
[SurQCT]:       Reading Grouping from File: /home/maitreyee/WORKSPACE//Air_Database/Run_0D/database/grouping/O3_UMN/O2/LevelsMap_DPM45.csv


In [16]:
def Save_RatesAtT_HDF5( NNRates, PathToHDF5File, TTra, TInt, KExcitType):
    PathToFile    = PathToHDF5File
    HDF5Exist_Flg = path.exists(PathToFile)
    f = h5py.File(PathToFile, 'a')
    
    TStr = 'RunI' + str(int(NNRunIdx)) + '/T_' + str(int(TTra)) + '_' + str(int(TInt)) + '/Rates/'       
    if TStr in f.keys():
        grp       = f[TStr]
        Data      = grp[KExcitType]
        Data[...] = NNRates

    else:
        grp       = f.create_group(TStr)
        Proc1     = grp.create_dataset(KExcitType, data=NNRates, compression="gzip", compression_opts=9)

    f.close()
    
def compute_BWDRates(KExcit_NN, TTra):
    Str = 'q_'+str(int(TTra))
    for iLevel in tqdm(range(NLevels[0]), desc='[SurQCT]:     Computing backward rates'):
        for jLevel in range(iLevel+1, NLevels[0]):
#             if (DiatData[1]['EInt'].to_numpy()[jLevel] > DiatData[0]['EInt'].to_numpy()[iLevel]):
            KExcit_NN[iLevel, jLevel] = KExcit_NN[jLevel, iLevel] * (DiatData[1][Str].to_numpy()[jLevel]) / (DiatData[1][Str].to_numpy()[iLevel])

    return KExcit_NN

In [18]:
KInel_NN_FWD = np.zeros((NLevels[0],NLevels[0]))
for TTran in TTranVec:
    Str = 'q_'+str(int(TTran))
    for iIdx in tqdm(range(NLevels[0]), desc='[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes'):
        time.sleep(0.02)
    
        if (InputData.ExoEndoFlg):
            jIdxVec           = [jIdx for jIdx in np.arange(NLevels[1]) if (DiatData[1]['EInt'].to_numpy()[jIdx] < DiatData[0]['EInt'].to_numpy()[iIdx])]
        else:
            jIdxVec           = [jIdx for jIdx in np.arange(NLevels[1]) if (DiatData[1][Str].to_numpy()[jIdx]    > DiatData[0][Str].to_numpy()[iIdx])]
        jNLevels              = len(jIdxVec)
    
        ### FWD Rates
        iiIdxVec              = [iIdx]*jNLevels
    
        TTranVec              = np.ones((jNLevels))*TTran
        TTranDataTemp         = pd.DataFrame({'TTran': TTranVec})
        TTranDataTemp.index   = jIdxVec

    
        iLevelsDataTemp       = LevelsData[0].iloc[iiIdxVec,:].copy()
        iLevelsDataTemp.index = jIdxVec

        jLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
        if (OtherVar == '_Delta'):
            jLevelsDataTemp   = iLevelsDataTemp.subtract(jLevelsDataTemp) 
        else:
            jLevelsDataTemp   = jLevelsDataTemp
        jLevelsDataTemp.index = jIdxVec
    
        kLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
        kLevelsDataTemp.index = jIdxVec

    
        iLevelsData           = pd.concat([iLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
        iLevelsData.columns   = [(VarName + '_i') for VarName in iLevelsData.columns]
    
        jLevelsData           = pd.concat([jLevelsDataTemp[xVarsVec_Delta], TTranDataTemp], axis=1)
        jLevelsData.columns   = [(VarName + OtherVar) for VarName in jLevelsData.columns]

        kLevelsData           = pd.concat([kLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
        kLevelsData.columns   = [(VarName + '_j') for VarName in kLevelsData.columns]
    
        xTemp_FWD             = pd.concat([iLevelsData, jLevelsData, kLevelsData], axis=1)
    
   
        if (len(xTemp_FWD[NN_KExcit.xTrainingVar]) > 0):
        
            if(ExcitType == 'KInel'):   
                KInel_NN_FWDTemp = np.exp( NN_KExcit.Model.predict(xTemp_FWD[NN_KExcit.xTrainingVar]) ) / InputData.MultFact         
                for jIdx in jIdxVec:
                    KInel_NN_FWD[iIdx, jIdx] = KInel_NN_FWDTemp[jIdx, 0]
            elif(ExcitType == 'KExch'):      
                KExch_NN_FWDTemp = np.exp( NN_KExcit.Model.predict(xTemp_FWD[NN_KExcit.xTrainingVar]) ) / InputData.MultFact
                # Exch matrix BWD and saving to HDF5 to be added
                
    if(ExcitType == 'KInel'):
        KExcit_NN = compute_BWDRates(KInel_NN_FWD.copy(), TTran)
        Save_RatesAtT_HDF5(KExcit_NN, InputData.PathtoHDF5NN, TTran, TTran, ExcitType)    
#     elif(ExcitType == 'KExch'):      
#         KExcit_NN = compute_BWDRates(KExch_NN_FWD, TTran)
#         Save_RatesAtT_HDF5(KExcit_NN, InputData.PathtoHDF5NN, TTran, TTran, ExcitType)
    

[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes: 100%|██████████| 6115/6115 [16:59<00:00,  6.00it/s]
[SurQCT]:     Computing backward rates: 100%|██████████| 6115/6115 [02:37<00:00, 38.71it/s] 


OSError: Unable to create file (unable to open file: name = '/home/maitreyee/WORKSPACE//Air_Database/HDF5_Database_NN/O3_UMN.hdf5', errno = 17, error message = 'File exists', flags = 15, o_flags = c2)